# **Setup Vector Store**

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [4]:
import os
api_key = os.environ.get("OPENAI_API_KEY")

In [5]:
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

/var/folders/pb/tqmxz14n6ps9b507bx6376bh0000gn/T/ipykernel_6828/3074274975.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")


In [8]:
current_dir = os.getcwd()
db_dir = os.path.join(current_dir, "store")  

db = Chroma(
    persist_directory=db_dir,
    embedding_function=embedding_model
)

# Chroma 컬렉션 정보 출력
print("Chroma 데이터베이스 컬렉션 확인:")
print(db._client.list_collections())

Chroma 데이터베이스 컬렉션 확인:
[Collection(name=langchain)]


In [9]:
# MMR 기반 검색 설정
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 1,
        "lambda_mult": 0.8
    }
)

In [10]:
# 검색 테스트 및 RAG 시스템에서 사용할 Context 생성
question = "빅뱅이론과 연대측정의 문제점이 무엇입니까?"
retrieved_docs = retriever.get_relevant_documents(question)
context = "\n".join([doc.page_content for doc in retrieved_docs])
print(context)

/var/folders/pb/tqmxz14n6ps9b507bx6376bh0000gn/T/ipykernel_6828/428189156.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


사실은 빅뱅 이론에 반대되는 실험 결과나 관측들은 종종 과학적 보고에 제한될 수 있음을 보여주는 것이다.[3] 그러나 이 결과는 전혀 기괴한 것이 아니며, 초기의 비교적 거칠었던 반양성자의 자기모멘트 측정 연구 결과를 다시 한번 확인해주는 것이었다. 빅뱅 이론의 예측과 달리, 이러한 측정은 실험 가능하고, 반복 가능한, 실제 과학이었다. 이 결과는 빅뱅 이론의 신봉자에게만 오직 기이한 것이다.빅뱅 이론으로는 우리의 우주가 존재하지 않아야한다는 결론을 얻은 것은 이번이 처음이 아니다.[4] 빅뱅 과학자들은 그들의 고집스런 주장을 계속하기 위해서는 신뢰할만한 증거들을 제시해야만 한다. 그러나 관측되는 증거들은 빅뱅 이론과 반대된다.빅뱅 과학자들은 빅뱅 이론에 반대되는 그 어떠한 관측 사실도 받아들이지 않는다.빅뱅이 사실이라면 우주가 존재해서는 안 된다는 결론조차도 말이다!빅뱅은 언제나 지지되지 않았던 추측이었다.왜 세속주의자들은 모든 것들의 존재를 창조주를 배제하고 설명하기 위해, 그러한 몽상적 개념에 그렇게도 필사적으로 집착하는 것일까? 그리고도대체 왜 그토록 많은 크리스천들이 하나님이 우주를 창조하기 위해 빅뱅을 사용하셨다고 주장하는 것일까? 특별히 이 과학적으로 파산한, 추정과 추측으로 이루어진 이론이 창세기의 기록과 많은 면에서 모순되는 데도 말이다.[5] 빅뱅 이론은 언제나 지지되지 않았던 추측이었다. 시간이 지남에 따라, 그 사실이 점점 더 분명해지고 있다.References1. Smorra, C. et al. 2017.A parts-per-billion measurement of the antiproton magnetic moment.Nature. 550 (7676): 371-374.2. Osborne, H.The Universe Should Not Actually Exist, CERN Scientists Discover. Newsweek. Posted on newsweek.com October 25, 2017, accessed October 26,


# **Augment**

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [22]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(
        "당신은 지적설계를 지지하는 과학자입니다. 당신의 청중은 창조과학을 공부하는 기독교 대학생입니다."
    ),
    HumanMessagePromptTemplate.from_template(
        """
        {question}
        아래의 문맥에 기반하여 답해주세요.
        {context}
        """
    )
])

In [23]:
message = chat_template.format_messages(
    question = question,
    context = context[0]
)

# **Generate**

In [24]:
from langchain_openai import ChatOpenAI

In [25]:
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0
)

In [26]:
for chunk in model.stream(message):
    print(chunk.content, end="", flush=True)

빅뱅 이론과 연대 측정의 문제점은 여러 가지가 있습니다. 이론적으로 우주의 기원과 진화를 설명하는 데 유용하지만, 몇 가지 한계와 의문점이 존재합니다.

1. **초기 상태의 불확실성**: 빅뱅 이론은 우주가 매우 높은 온도와 밀도로 시작되었다고 주장합니다. 그러나 그 초기 상태에 대한 구체적인 이해는 부족합니다. 초기 우주의 물리적 조건을 정확히 알 수 없기 때문에 이론의 기초가 되는 가정들이 의문을 제기받을 수 있습니다.

2. **관측의 한계**: 우주의 초기 상태를 직접 관측할 수 있는 방법이 제한적입니다. 예를 들어, 우주 배경 복사(CMB)는 빅뱅 이론을 지지하는 중요한 증거로 여겨지지만, 이 복사가 발생한 시점 이후의 우주에 대한 정보만을 제공합니다. 따라서 초기 우주에 대한 직접적인 증거가 부족합니다.

3. **연대 측정의 불확실성**: 우주의 나이를 측정하는 방법은 여러 가지가 있지만, 각 방법마다 오차가 존재합니다. 예를 들어, 별의 진화나 우주 팽창 속도에 기반한 측정은 서로 다른 결과를 낼 수 있으며, 이는 우주의 나이에 대한 불확실성을 초래합니다.

4. **다양한 해석 가능성**: 빅뱅 이론은 여러 가지 해석이 가능하며, 이는 과학적 논의의 여지를 남깁니다. 예를 들어, 우주의 팽창 속도나 물질의 분포에 대한 다양한 모델이 존재하며, 이로 인해 이론의 해석이 달라질 수 있습니다.

5. **대안 이론의 존재**: 빅뱅 이론 외에도 우주의 기원과 진화를 설명하려는 다양한 대안 이론들이 존재합니다. 이러한 이론들은 빅뱅 이론이 설명하지 못하는 부분을 보완하려고 하며, 이는 과학적 논의의 폭을 넓히는 동시에 빅뱅 이론의 절대성을 의문시할 수 있습니다.

이러한 문제점들은 빅뱅 이론과 연대 측정이 완벽하지 않음을 보여주며, 과학적 탐구의 지속적인 필요성을 강조합니다. 창조과학의 관점에서 볼 때, 이러한 한계는 우주와 생명의 기원을 이해하는 데 있어 지적 설계의 가능성을 제시할 수 있습니다.